In [ ]:
!pip install -U sentence_transformers

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import torch
from tqdm import tqdm

In [59]:
"/kaggle/input/ml-challenge/dataset"
train = pd.read_csv('/kaggle/input/ml-challenge/dataset/train.csv', nrows=100_000)
# train = train.loc[train["PRODUCT_LENGTH"] < 1_5000]
train.fillna('', inplace=True)
# embed2 = pd.read_csv('/kaggle/input/ml-challenge/dataset/train.csv')

In [60]:
train['COMBINED'] = train['TITLE'] + train['DESCRIPTION'] + ['BULLET_POINTS']
train.drop(columns=['TITLE','DESCRIPTION', 'BULLET_POINTS'], inplace=True)
train['COMBINED'] = train['COMBINED'].str.lower()

In [ ]:
train.head()

In [ ]:
from nltk.corpus import stopwords

sample_series = train['COMBINED'].iloc[60_000:200_000]

def remove_stop_words(text):
    stop_words = set(stopwords.words('english'))
    words = text.split()
    result_words = [word for word in words if word.lower() not in stop_words]
    result_text = ' '.join(result_words)
    return result_text

train['COMBINED'] = sample_series.apply(remove_stop_words)

sample.head()

In [61]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
from sklearn.metrics import mean_absolute_percentage_error as mape
from torch.utils.data import DataLoader
import torch

start = 60_000
end = 70_000
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')


model.classifier = torch.nn.Linear(in_features=768, out_features=1)

# Prepare your data
train_texts = train['COMBINED'].iloc[start:end].tolist()
train_labels = train['PRODUCT_LENGTH'].iloc[start:end].values
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
train_labels = torch.tensor(train_labels)

class TextRegressionDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = TextRegressionDataset(train_encodings, train_labels)
train_loader = DataLoader(train_dataset, batch_size=16)


device = torch.device('cuda')
model.to(device)
state_dict = torch.load('/kaggle/working/weights.pt')
model.load_state_dict(state_dict)
model.train()

optim = torch.optim.Adam(model.parameters(), lr=5e-5)

for batch in tqdm(train_loader):
    optim.zero_grad()
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    labels = batch['labels'].to(device).unsqueeze(1)
    outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
    loss = outputs.loss
    loss.backward()
    optim.step()

torch.save(model.state_dict(), 'weights.pt')
print('model saved')
    
    

In [75]:
start = 60_000
end = 70_000

texts = train['COMBINED'][start: end].tolist()
train_encodings = tokenizer(train_texts, truncation=True, padding=True)

model.eval()

inputs = tokenizer(texts, return_tensors='pt', padding=True, truncation=True)
input_ids = inputs['input_ids']
attention_mask = inputs['attention_mask']

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
input_ids = input_ids.to(device)
attention_mask = attention_mask.to(device)

with torch.no_grad():
    output = model(input_ids, attention_mask=attention_mask)

y_pred = output.logits.detach().cpu().numpy()
print(y_pred.shape)
    
y_test = train['PRODUCT_LENGTH'][start: end].values
print(y_test.shape)

from sklearn.metrics import mean_absolute_percentage_error as mape
error = mape(y_test, y_pred)
print(f'error: {error}')
score = max(0, 100 * (1 - error))
print(f'final score: {score}')

OutOfMemoryError: CUDA out of memory. Tried to allocate 14.65 GiB (GPU 0; 15.90 GiB total capacity; 1.40 GiB already allocated; 10.00 GiB free; 5.17 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF